In [1]:
from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv
from ot2_env_wrapper import OT2Env  # Custom environment wrapper
from clearml import Task  # Import ClearML's Task
import typing_extensions
from stable_baselines3.common.env_util import DummyVecEnv
from stable_baselines3 import PPO  # Replace with your model type
# Paths to your saved models
model_paths = [
    r"C:\Users\Edopi\Downloads\model (5).zip"
]

def test_model(model_path, env):
    # Load the trained model
    model = PPO.load(model_path)
    # Reset the environment
    obs = env.reset()
    if isinstance(obs, tuple):  # Handle tuple from new Gym API
        obs = obs[0]

    print("Initial observation shape:", obs.shape)

    done = False
    total_distance = []

    while not done:
        if len(obs.shape) > len(env.observation_space.shape):  # Check for extra dimensions
            obs = obs.squeeze()

        if obs.shape != env.observation_space.shape:  # Adjust shape if necessary
            print(f"Adjusting observation shape from {obs.shape} to {env.observation_space.shape}")
            obs = obs[:env.observation_space.shape[0]]

        print("Processed observation shape:", obs.shape)

        # Predict the action
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)

        if isinstance(obs, tuple):  # Handle tuple from Gym API in step
            obs = obs[0]

        # Process results as needed
        total_distance.append(info.get("distance", 0))

    avg_distance = np.mean(total_distance)
    std_distance = np.std(total_distance)

    return avg_distance, std_distance

def main():
    # Wrap the environment
    env = DummyVecEnv([lambda: Monitor(OT2Env(max_steps=1000))])

    results = {}

    # Test each model
    for model_path in model_paths:
        print(f"Testing model: {model_path}")
        avg_distance, std_distance = test_model(model_path, env)
        results[model_path] = {"avg_distance": avg_distance, "std_distance": std_distance}
        print(f"Model: {model_path}, Avg Distance: {avg_distance:.4f}, Std Dev: {std_distance:.4f}")

    # Identify the best model
    best_model = min(results, key=lambda x: results[x]["avg_distance"])
    print(f"\nBest Model: {best_model}")
    print(f"Performance: Avg Distance = {results[best_model]['avg_distance']:.4f}, Std Dev = {results[best_model]['std_distance']:.4f}")

    env.close()

if __name__ == "__main__":
    main()


Testing model: C:\Users\Edopi\Downloads\model (5).zip
Reset: Pipette Position [0.073  0.0895 0.1195], Goal Position [ 0.05497801 -0.06764789  0.19775745]
Initial observation shape: (1, 9)
Processed observation shape: (9,)


ValueError: Error: Unexpected observation shape (9,) for Box environment, please use (6,) or (n_env, 6) for the observation shape.

In [11]:
env = DummyVecEnv([lambda: Monitor(OT2Env(max_steps=1000))])
obs, _ = env.reset()

Reset: Pipette Position [0.073  0.0895 0.1195], Goal Position [ 0.21544941 -0.00916656  0.18965301]


ValueError: could not broadcast input array from shape (9,) into shape (6,)

In [ ]:
import matplotlib.pyplot as plt

# Example visualization code
model_names = list(results.keys())
avg_distances = [results[model]["avg_distance"] for model in model_names]

plt.barh(model_names, avg_distances)
plt.xlabel("Average Distance to Target")
plt.ylabel("Model")
plt.title("Model Performance Comparison")
plt.show()
